# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [54]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [55]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [56]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [57]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modelling data based on required queries. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [58]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [59]:
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS event_data
    WITH REPLICATION = 
    {'class':'SimpleStrategy', 'replication_factor':1}
    """
    )
except Exception as e:
    print (e)

#### Set Keyspace

In [60]:
try:
    session.set_keyspace('event_data')
except Exception as e:
    print (e)

## Required queries of the data

### 1. Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [61]:
# File from which the data will be modelled from
file = 'event_datafile_new.csv'

### Query Model 1

##### Example query: Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### Queries will be done against the sessionId and itemInSession

In [63]:
# Create appropriate table based on the query
query1 = '''
    CREATE TABLE IF NOT EXISTS session_and_itemInSession_library 
    (session_id int, item_in_session int, artist text, song_title text, song_length float, 
    PRIMARY KEY (session_id, item_in_session)
    )
'''
try:
    session.execute(query1)
except Exception as e:
    print (e)

In [66]:
# Insert the data into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_and_itemInSession_library (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [67]:
# Verify that the data was entered into the table
query1 = """
    SELECT artist, song_title, song_length
    FROM session_and_itemInSession_library 
    WHERE session_id=338
    AND item_in_session=4
"""
try:
    results = session.execute(query1)
    df = pd.DataFrame(list(results))
    print (df)
except Exception as e:
    print (e)

      artist                       song_title  song_length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


In [68]:
# Validating CQL model & query produce same results as Pandas filtering
data1 = pd.read_csv(file)
data1[(data1['sessionId'] == 338) & (data1['itemInSession'] == 4)][['artist','song','length']]

,artist,song,length
5810,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query Model 2

##### Example query: Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Queries will be done against the userId and sessionId

In [69]:
# Create appropriate table based on the query
query2 = '''
    CREATE TABLE IF NOT EXISTS user_and_session_library 
    (user_id int, session_id int, item_in_session int, artist text, song_title text, song_length float, first_name text, last_name text, 
    PRIMARY KEY ((user_id, session_id), item_in_session)
    )
'''
try:
    session.execute(query2)
except Exception as e:
    print (e)                 

In [70]:
# Insert the data into the table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_and_session_library (user_id, session_id, item_in_session, artist, song_title, song_length, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], float(line[5]), line[1], line[4]))

In [71]:
# Verify that the data was entered into the table
query2 = """
    SELECT artist, song_title, first_name, last_name
    FROM user_and_session_library 
    WHERE user_id=10
    AND session_id=182
"""
try:
    results = session.execute(query2)
    df = pd.DataFrame(list(results))
    print (df)
except Exception as e:
    print (e)

              artist                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


In [72]:
# Validating CQL model & query produce same results as Pandas filtering
data2 = pd.read_csv(file)
data2[(data2['userId'] == 10) & (data2['sessionId'] == 182)][['artist','song','firstName','lastName']]

,artist,song,firstName,lastName
6805,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
6806,Three Drives,Greece 2000,Sylvie,Cruz
6807,Sebastien Tellier,Kilometer,Sylvie,Cruz
6808,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query Model 3

##### Example query: Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### Queries will be done against the song title, but the primary key will be a composite of the song title and the userId

In [83]:
# Create appropriate table based on the query
query3 = '''
    CREATE TABLE IF NOT EXISTS song_library 
    (song_title text, user_id int, first_name text, last_name text,  
    PRIMARY KEY ((song_title), user_id)
    )
'''
try:
    session.execute(query3)
except Exception as e:
    print (e)           

In [84]:
# Insert the data into the tables
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [85]:
# Verify that the data was entered into the table
query3 = """
    SELECT first_name, last_name
    FROM song_library 
    WHERE song_title='All Hands Against His Own'
"""
try:
    results = session.execute(query3)
    df = pd.DataFrame(list(results))
    print (df)
except Exception as e:
    print (e)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


In [86]:
# Validating CQL model & query produce same results as Pandas filtering
data3 = pd.read_csv(file)
data3[data3['song'] == 'All Hands Against His Own'][['firstName','lastName']]

,firstName,lastName
1313,Jacqueline,Lynch
1839,Tegan,Levine
5282,Sara,Johnson


### Drop the tables before closing out the sessions

In [82]:
table1 = 'session_and_itemInSession_library'
drop_query1 = "drop table " + table1
try:
    rows = session.execute(drop_query1)
    print (table1 + ' table has been dropped')
except Exception as e:
    print(e)

table2 = 'user_and_session_library'
drop_query2 = "drop table " + table2
try:
    rows = session.execute(drop_query2)
    print (table2 + ' table has been dropped')
except Exception as e:
    print(e)
    
table3 = 'song_library'
drop_query3 = "drop table " + table3
try:
    rows = session.execute(drop_query3)
    print (table3 + ' table has been dropped')
except Exception as e:
    print(e)

session_and_itemInSession_library table has been dropped
user_and_session_library table has been dropped
song_library table has been dropped


### Close the session and cluster connection¶

In [53]:
session.shutdown()
cluster.shutdown()